# Scaling Relations for Supported Graphene Project
***

# Notebook Setup

## Import Modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

In [1]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "data"))

sys.path.insert(0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow"))

# Python Modules **************************************************************
# *****************************************************************************
import numpy as np

import plotly.graph_objs as go
import plotly.plotly as py


# My Modules ******************************************************************
# *****************************************************************************
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot, Scaling_Relations_Plot

# Local Imports ***************************************************************
# *****************************************************************************
from an_data_processing import load_df, load_vegge_data

from proj_data_fe_graph import (
    system_color_map,
    smart_format_dict,    
    system_color_map_vegge,
    smart_format_dict_volcano_scaling,
    )

## Script Inputs

In [2]:
save_plot = False

smart_format_dict = smart_format_dict_volcano_scaling

In [3]:
# smart_format_dict

# Load Data

## Norskov DFT Data

In [4]:
%%capture
df_m = load_df(
    from_file=False,
    root_dir=os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow/adsorption_study"),
    filter_columns=[
        "pdos_data",
        "charge_history",
        "magmom_history",
        ],
    )

## Vegge DFT Data

In [5]:
from an_data_processing import load_vegge_data

df_vegge = load_vegge_data()

# Instantiate the ORR_Free_E_Plot Instance

In [6]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    smart_format=smart_format_dict,
    bias=0.,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=True,
    rxn_type="ORR",
    )

# Process ORR Adsorbate Data

## Norskov Data (Mine)

In [7]:
prop_name_list = [
    "author_short",
    "system",
    "site",
    "spinpol",
    ]

grouped = df_m.groupby([    
    "system",
    "site",
    "spinpol",
    ])

for i_ind, (name, df_i) in enumerate(grouped):
    ORR_PLT.add_series(
        df_i,
        plot_mode="all",
        overpotential_type="OER",
        property_key_list=prop_name_list,
        add_overpot=False,
        )

There is a nan in the energy dict!!!
*OOH energy set by *OH and standard scaling
There is a nan in the energy dict!!!
*OH energy set by *OOH and standard scaling


## Vegge ACS Data

In [8]:
prop_name_list = [
    "author_short",
    "system",
    "n_per_ads",
    "graphene_layers",
     ]

group_params = [
    "system",
    "n_per_ads",
    "graphene_layers",
    ]

grouped = df_vegge.groupby(group_params)
for i_ind, (name, df_i) in enumerate(grouped):
    system_i = "_".join([
        str(df_i.iloc[0]["system"]),
        str(df_i.iloc[0]["n_per_ads"]),
        str(df_i.iloc[0]["graphene_layers"])])

    ORR_PLT.add_series(
        df_i, plot_mode="all", group="vegge",
        overpotential_type="ORR",
        property_key_list=prop_name_list,
        color=system_color_map_vegge[system_i])

In [9]:
# grouped = df_vegge.groupby(group_params)
# for i_ind, (name, df_i) in enumerate(grouped):
#     system_i = "_".join([
#         str(df_i.iloc[0]["system"]),
#         str(df_i.iloc[0]["n_per_ads"]),
#         str(df_i.iloc[0]["graphene_layers"])])
#     print(system_i)

# Scaling Relations Class

## Instantiate Scaling Relations Instance

In [10]:
SC_PLT = Scaling_Relations_Plot(
    ORR_PLT,
    plot_range={
        "y": [0., 6.],
        "x": [-0.3, 3.5]})
    
slope_intercept_dict_ooh = SC_PLT.fit_scaling_lines("ooh")
slope_intercept_dict_o = SC_PLT.fit_scaling_lines("o")

Scaling fit for  ooh
intercept_i:  3.019770458693692
slope_i:  0.905043632644589

Scaling fit for  o
intercept_i:  0.8210146890807739
slope_i:  1.1892490228015251



## Add Fit Scaling Lines

In [11]:
SC_PLT.add_line(slope_intercept_dict_ooh,
    name="*OOH vs *OH Scaling",
    color="orange",
    width=1,
    dash="solid",
    )

SC_PLT.add_line(slope_intercept_dict_o,
    name="*O vs *OH Scaling",
    color="red",
    width=1,
    dash="solid",
    )

## Add ideal scaling, create plot traces, and collect data

In [12]:
SC_PLT.add_ideal_lines()

SC_PLT.create_scaling_relations_plot(
    smart_format_dict=smart_format_dict,
    )

data_m = SC_PLT.data_points["ooh_vs_oh"] + \
    SC_PLT.data_points["o_vs_oh"] + \
    SC_PLT.data_points["oh_vs_oh"] + \
    SC_PLT.data_lines

# Plotting

## Plot Layout Settings

In [13]:
tick_lab_size = 8 * (4. / 3.)
axes_lab_size = 9 * (4. / 3.)

layout_override = {

    "width": 1.5 * 7.964 * 37.795275591,
    "height": 1.5 * 5.6002 * 37.795275591,

    "yaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },


    "xaxis": {
        "titlefont": {
            "size": axes_lab_size,
            },
        "tickfont": {
            "size": tick_lab_size,
            },
        },
    
    "margin": go.layout.Margin(
        b=50.,
        l=50.,
        r=50.,
        t=50.,        
        ), 
    }

layout = SC_PLT.__create_layout__(
    title=None,
    showlegend=False,
    layout_dict=layout_override,
    )
layout["annotations"] = SC_PLT.annotations_list

## Make Plot | No Legend/Small

In [14]:
# save_plot = False
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

fig = dict(data=data_m, layout=layout)
py.iplot(fig, filename=os.path.join(save_dir, "pl_supp_graph_scaling_relations"))

## Make Plot | w/ Legend/Big

In [15]:
layout = SC_PLT.__create_layout__(
#     x_ax_spec="oh",
    title="Scaling Relations",
    showlegend=True,
    )

layout["margin"] = go.layout.Margin(
    b=50.,
    l=50.,
    r=50.,
    t=50.,
#     pad=20.,
    )

layout["width"] = 4.5 * 7.964 * 37.795275591
# layout["height"] = 5.5 * 5.6002 * 37.795275591
layout["height"] = 4. * 5.6002 * 37.795275591

layout["annotations"] = SC_PLT.annotations_list


# save_plot = False
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

fig = dict(data=data_m, layout=layout)
py.iplot(
    fig,
    filename=os.path.join(save_dir, "pl_supp_graph_scaling_relations_big"))